<a href="https://colab.research.google.com/github/JoyceLarona/Portfolio/blob/main/Market_Lookup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Identifying the nearest Market Lookup

---

## Problem Statement

> The problem statement is if there is a bulk way to identify the correct market lookup and market density zone

---

## Background
> This guide explains how to use the lat/long of the SF building data to find the nearest market and calculate the distance.

---

## Requirements

> Gather/Prepare the following Excel files:

>> france_all.xlsx >> SF Building Records

>> france_ml.xlsx >> France market lookups (12 cities)

---

## Language

> Python

---

## Tools

> Google Colab

---

## Methodology

> Follow the detailed instructions in the preceding codes

*   Open Google Colab in your web browser
> https://colab.research.google.com/
*   Sign in with your google account
*   Upload the gathered files in Google Colab - See "Uploading Files in Google Colab"
*   You can also use the files uploaded in Google Drive by connecting it - See "Connecting Google Drive for the files"
*   Copy and paste the code in the preceding steps

> Uploading Files in Google Colab

* Click Files in left side of the screen
* Click upload
* Select the document you want to upload

> Connecting Google Drive for the files

* Copy and paste below code
```
from google.colab import drive
drive.mount('/content/drive')
```

> Import panda library

* Copy and paste below code
```
import pandas as pd
```

> Pandas read excel

* You can change the directory of the file
* You can the path by clicking 3 dots on the right of the file and select "Copy path"
* Copy and paste below code
```
cities = pd.read_excel('/content/drive/MyDrive/france_all.xlsx')
market = pd.read_excel('/content/drive/MyDrive/france_ml.xlsx')
```

> Creating a new Excel file with added columns

In the provided code below, the geodesic function from the geopy.distance module is used to calculate the distance between two sets of latitude and longitude coordinates. The geodesic function returns the geodesic distance between two points on the surface of an ellipsoidal model of the Earth.

Here, for each row in the GeoDataFrame gdf_cities, the calculate_distances function iterates over the rows of gdf_market and calculates the geodesic distance using the geodesic function. The distances are stored in the distances dictionary, and the nearest city is determined based on the minimum distance.

Please note that this code calculates the distance for each city to all the market cities and finds the nearest market city for each city in the GeoDataFrame. The distances are then stored in a new DataFrame distances_df and concatenated to the original GeoDataFrame gdf_cities.


```
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic

# Convert the DataFrames to GeoDataFrames with Point geometries
geometry_cities = [Point(lon, lat) for lon, lat in zip(cities['Longitude'], cities['Latitude'])]
gdf_cities = gpd.GeoDataFrame(cities, geometry=geometry_cities, crs='EPSG:4326')

geometry_market = [Point(lon, lat) for lon, lat in zip(market['long1'], market['lat1'])]
gdf_market = gpd.GeoDataFrame(market, geometry=geometry_market, crs='EPSG:4326')

# Check and drop invalid and empty geometries in both GeoDataFrames
gdf_cities = gdf_cities[gdf_cities.geometry.is_valid & ~gdf_cities.geometry.is_empty]
gdf_market = gdf_market[gdf_market.geometry.is_valid & ~gdf_market.geometry.is_empty]

# Function to calculate distances for a single row and find the nearest city
def calculate_distances(row):
    distances = {}
    min_distance = float('inf')
    nearest_city = None

    for market_idx in gdf_market.index:
        market_city = gdf_market.loc[market_idx, 'city']
        distance = geodesic((row['Latitude'], row['Longitude']),
                            (gdf_market.loc[market_idx, 'lat1'], gdf_market.loc[market_idx, 'long1'])).kilometers
        distances[market_city] = distance

        if distance < min_distance:
            min_distance = distance
            nearest_city = market_city

    distances['nearest_city'] = nearest_city
    return distances

# Apply the function to each row in the GeoDataFrame
results = gdf_cities.apply(calculate_distances, axis=1)

# Convert the list of dictionaries to a DataFrame
distances_df = pd.DataFrame(results.tolist(), index=gdf_cities.index)

# Concatenate the distances DataFrame to the original GeoDataFrame
gdf_cities = pd.concat([gdf_cities, distances_df], axis=1)

# Save the resulting GeoDataFrame to a new Excel file
gdf_cities.to_excel('france_market.xlsx', index=False)
```

> Coloring the cells with nearest distance and create a new excel file

```
import openpyxl
from openpyxl.styles import PatternFill

# Load the Excel file
excel_file = 'france_market.xlsx'
workbook = openpyxl.load_workbook(excel_file)
sheet = workbook.active

# Iterate through rows and highlight the cell with the lowest distance in columns L to W
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=12, max_col=23):  # Assuming distances are in columns L to W
    min_distance_cell = min(row, key=lambda cell: float('inf') if cell.value is None or (isinstance(cell.value, str) and not cell.value.replace('.', '', 1).isdigit()) else float(cell.value))
    min_distance_cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")  # aRGB hex for yellow

# Save the changes
workbook.save(excel_file)
```

